# Examples of utils provided by ReGraph's Neo4jHierarchy module

In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [3]:
# here we clear the hierarchy
h.clear()
h.drop_all_constraints()

## Hierarchy

A hierarchy is represented by at least 2 graphs with a typing relation (homorphism) between them.


Each graph in the database is represented by a unique label which enables us to match quickly the nodes of this graph. Each node of a graph has then the labels *node* and *graphLabel*. We can match these nodes by using the query:

> MATCH (n:node:graphLabel) RETURN n


The edges of a graph are labeled as *edge* and the typing edges are labeled as *typing*. We can then easily find the image of a node with the query:

> OPTIONAL MATCH (n:node:graphLabel)-[:typing]->(m) RETURN m



## Adding a graph to the hierarchy (Action Graph)

Here we create a first graph called *ActionGraph*. This graph represent people and the relationship between them.

In [4]:
h.add_graph('ActionGraph')

In [5]:
ag = h.access_graph('ActionGraph')

In [6]:
nodes = [
    ("a", {"name": "Jack", "age": 23, "hobby": {"hiking", "music"}, "weight": 75}), 
    ("b", {"name": "Bob", "age": 24, "hobby": {"sport", "music"}, "height": 178}),
    "c", 
    ("d", {"name": "Paul"}), "e", "f"
]
edges = [
    ("a", "b", {"type": {"friends", "colleagues"}}), 
    ("d", "b", {"type": "enemies"}), 
    ("a", "c"),
    ("d", "a", {"type": "friends"}),
    ("e", "a"), 
    ("f", "d")
]
ag.add_nodes_from(nodes)
ag.add_edges_from(edges)

## Adding a second graph to the herarchy (Meta Model)

Here we create a second graph called *MetaModel*. Which represent the types of people (*Adult* or *Child*) and the relationships between them.

In [7]:
h.add_graph('MetaModel')

In [8]:
mm = h.access_graph('MetaModel')

In [9]:
nodes = [
    ("a", {"type": "Adult"}),
    ("b", {"type": "Child"})
]
edges = [
    ("a", "a", {"type": {"friends", "colleagues", "enemies"}}),
    ("a", "b", {"type": {"friends", "colleagues", "enemies"}}),
    ("b", "a", {"type": {"friends", "colleagues", "enemies"}}),
    ("b", "b", {"type": {"friends", "colleagues", "enemies"}})
]
mm.add_nodes_from(nodes)
mm.add_edges_from(edges)

## Typing

Here we create a typing of the *ActionGraph* on the *MetaModel*.

In [10]:
mapping = {
    "a":"a",
    "b":"a",
    "c":"b",
    "d":"b",
    "e":"b",
    "f":"b"
}
h.add_typing('ActionGraph', 'MetaModel', mapping)

## Classic update functions

We still can use the modification functions on these grapphs.

In [10]:
ag.clone_node("a")

## Removing a graph from the hierarchy

We can also remove a graph from the hierarchy. All its nodes and connections (edges and typing edges) are removed.

In [11]:
h.remove_graph('MetaModel')